This notebook takes ground truth data for tracking and converts it to the OpenPTV data in Frames

In [1]:
import pathlib, os
import re
import numpy as np
import pandas as pd
import os


In [2]:
working_directory = pathlib.Path('/home/user/Dropbox/Open_Pro_My_PTV/Tracking/949_particles')
os.chdir(pathlib.Path(working_directory))
print(os.path.abspath(os.getcwd()))

/home/user/Dropbox/Open_Pro_My_PTV/Tracking/949_particles


In [3]:
def read_origin_file(filepath):
    """
    Reads an OpenPTV origin file and returns a pandas DataFrame.
    """
    # Read the header line
    with open(filepath) as f:
        for line in f:
            if line.startswith('#'):
                header = line[1:].strip().split()
                break

    # Read the data, skipping the header line(s)
    df = pd.read_csv(
        filepath,
        comment='#',
        sep=r'\s+',
        names=header
    )
    df.rename(columns=lambda x: x.strip(','), inplace=True)
    df['ID'] = df['ID'].astype(int)

    for col in df.columns:
        if col != 'ID':
            df[col] = df[col].astype(float)


    # Move 'X' column to the end

    # Rename columns as specified
    df = df.rename(columns={'Y': 'X', 'Z': 'Y', 'X': 'Z'})

    return df

origin_df = read_origin_file(working_directory / 'origin' / 'origin_00000.txt')
origin_df.head()

,ID,Z,X,Y,xc0,yc0,xc1,yc1,xc2,yc2,xc3,yc3
0,250,134.554431,165.677572,143.637106,375.044450,356.012807,384.389545,416.088849,439.233023,337.090519,426.958841,418.681389
1,695,154.365276,146.095693,130.049090,339.753993,388.769437,348.519439,441.318202,395.954569,368.374971,382.727308,445.828540
2,1069,154.059499,134.594652,142.787111,316.213915,362.755296,325.009035,413.825967,372.656106,342.513880,358.629590,419.984701
3,1741,141.145272,148.266317,148.025512,341.389654,348.856569,350.494637,405.163203,402.908139,329.452818,389.612592,409.861142
4,2153,147.940949,138.090230,127.653911,321.729748,392.305740,330.123325,446.380785,381.398907,371.366818,367.769572,451.752579


In [4]:
def load_all_origin_files(origin_dir):
    """
    Loops through all files in the given origin_dir, reads each file using read_origin_file,
    and returns a list of (frame_number, DataFrame) tuples sorted by frame_number.
    """
    origin_files = sorted(pathlib.Path(origin_dir).glob('origin_*.txt'))
    all_origin_data = []
    for file in origin_files:
        # Extract frame number from filename, e.g., origin_00012.txt -> 12
        match = re.search(r'origin_(\d+)\.txt', file.name)
        if match:
            frame_number = int(match.group(1))
            df = read_origin_file(file)
            all_origin_data.append((frame_number, df))
    # Sort by frame_number
    all_origin_data.sort(key=lambda x: x[0])
    return all_origin_data

# Example usage:
all_origin_data = load_all_origin_files(working_directory / 'origin')

In [5]:
def write_all_openptv_files(all_origin_data, output_dir_img='particle_images', output_dir_res='res'):
    """
    For each (frame_number, DataFrame) in all_origin_data, write OpenPTV-style files
    with filenames adjusted to the frame number.
    2D files go to output_dir_img/c[cam]/ as c[cam].[frame]_targets
    3D files go to output_dir_res as rt_is.[frame]
    """

    os.makedirs(output_dir_img, exist_ok=True)
    os.makedirs(output_dir_res, exist_ok=True)

    for cam in range(4):
        os.makedirs(os.path.join(output_dir_img, f'c{cam}'), exist_ok=True)

    for frame_number, origin_df in all_origin_data:
        num_particles = len(origin_df)
        df_3d = pd.DataFrame({
            'index': np.arange(1, num_particles + 1),  # 1-based
            'X': origin_df['X'],
            'Y': origin_df['Y'],
            'Z': origin_df['Z'],
        })

        # 2D DataFrames for each camera
        dfs_2d = []
        for cam in range(4):
            df_2d = pd.DataFrame({
                'target_idx': np.arange(num_particles),  # 0-based
                'x': origin_df[f'xc{cam}'],
                'y': origin_df[f'yc{cam}'],
                'npix': 3,
                'nx': 3,
                'ny': 3,
                'sumg': 10,
                '3d_pointer': np.arange(1, num_particles + 1)  # 1-based pointer to 3D row
            })
            dfs_2d.append(df_2d)

        # 3D pointers to 2D
        pointers = []
        for i, row in origin_df.iterrows():
            p = []
            for cam in range(4):
                if pd.isna(row[f'xc{cam}']) or pd.isna(row[f'yc{cam}']):
                    p.append(-1)
                else:
                    p.append(i)
            pointers.append(p)
        df_3d[['p0', 'p1', 'p2', 'p3']] = pd.DataFrame(pointers, index=df_3d.index)

        # Write 2D files in subfolders
        for cam, df_2d in enumerate(dfs_2d):
            cam_folder = os.path.join(output_dir_img, f'c{cam}')
            filename = os.path.join(cam_folder, f"c{cam}.{frame_number:04d}_targets")
            with open(filename, "w") as f:
                f.write(f"{len(df_2d)}\n")
                for _, row in df_2d.iterrows():
                    f.write("{:4d} {:9.4f} {:9.4f} {:5d} {:5d} {:5d} {:5d} {:5d}\n".format(
                        int(row['target_idx']),
                        row['x'], row['y'],
                        int(row['npix']), int(row['nx']), int(row['ny']),
                        int(row['sumg']), int(row['3d_pointer'])
                    ))

        # Write 3D file
        filename_3d = os.path.join(output_dir_res, f"rt_is.{frame_number}")
        with open(filename_3d, "w") as f:
            f.write(f"{len(df_3d)}\n")
            for _, row in df_3d.iterrows():
                f.write("{:4d} {:9.4f} {:9.4f} {:9.4f} {:5d} {:5d} {:5d} {:5d}\n".format(
                    int(row['index']),
                    row['X'], row['Y'], row['Z'],
                    int(row['p0']), int(row['p1']), int(row['p2']), int(row['p3'])
                ))

write_all_openptv_files(all_origin_data)